![Hormone Lab Hero](assets/hormone_hero.png)

# 💍 The Bio-Active Hormone Lab: Peptide Drug Discovery 🧬

**Objective**: Learn how to model real-world therapeutic macrocycles like **Oxytocin** and **Cyclosporine A** using physics-based generation.

---

### 🌟 What is a Macrocycle?
In drug discovery, a **macrocycle** is a molecule containing a large ring (usually 12 or more atoms). Unlike linear peptides which are floppy and easily destroyed by the body, macrocycles are **pre-organized** into specific shapes. This makes them:
1. **Potent**: They fit into their target protein like a perfect key.
2. **Stable**: Their circular structure protects them from being chopped up by enzymes.

In this lab, we use `synth-pdb` to find the "Native" physical conformation of these therapeutic giants.

In [ ]:
# @title Setup & Installation { display-mode: "form" }
import os
import sys
from pathlib import Path

try:
    current_path = Path(".").resolve()
    repo_root = current_path.parent.parent 
    if (repo_root / "synth_pdb").exists():
        if str(repo_root) not in sys.path:
            sys.path.insert(0, str(repo_root))
            print(f"📌 Added local library to path: {repo_root}")
except Exception:
    pass

if 'google.colab' in str(get_ipython()):
    if not os.path.exists("installed.marker"):
        print("Running on Google Colab. Installing dependencies...")
        get_ipython().run_line_magic('pip', 'install synth-pdb py3Dmol biotite numpy matplotlib')
        
        with open("installed.marker", "w") as f:
            f.write("done")
        
        print("🔄 Installation complete. KERNEL RESTARTING AUTOMATICALLY...")
        os.kill(os.getpid(), 9)
    else:
        print("✅ Dependencies Ready.")
else:
    import synth_pdb
    print(f"✅ Running locally. Using synth-pdb version: {synth_pdb.__version__}")

In [ ]:
import numpy as np
import py3Dmol
from synth_pdb.generator import generate_pdb_content
from synth_pdb.batch_generator import BatchedGenerator

print("Molecular Pharmacopoeia: ONLINE 🧪")

## 1. Case Study: Oxytocin (The Disulfide Loop)

**Oxytocin** is a 9-residue peptide hormone. It is held in its active shape by a **disulfide bridge** between its first and sixth residues (`Cys1` and `Cys6`). 

**The Challenge**: To close a disulfide loop, the two Sulfur atoms must come within ~2.0Å of each other. In a random linear chain, this is nearly impossible. We use **physics-based minimization** to "pull" the ring shut.

In [ ]:
# Sequence: CYS-TYR-ILE-GLN-ASN-CYS-PRO-LEU-GLY
oxytocin_seq = "CYS-TYR-ILE-GLN-ASN-CYS-PRO-LEU-GLY"

print("💍 Synthesizing Oxytocin with Disulfide Closure...")

# We generate the structure and enable minimization to resolve the loop
oxy_pdb = generate_pdb_content(
    sequence_str=oxytocin_seq, 
    minimize_energy=True, 
    cap_termini=True
)

print("✅ Synthesis Complete.")

In [ ]:
import py3Dmol
view = py3Dmol.view(width=800, height=500)
view.setBackgroundColor('#fdfdfd')
view.addModel(oxy_pdb, 'pdb')

# 1. Style the main backbone with ribbons
view.setStyle({'model': -1}, {'cartoon': {'color': '#a29bfe', 'opacity': 0.9}})

# 2. STYLE THE BRIDGE: Show Cys side chains as thick sticks
# Use addStyle to layer the Cys representation without breaking connectivity.
view.addStyle({'resn': 'CYS'}, {'stick': {'radius': 0.2, 'color': '#fdcb6e'}})

# 3. Highlight the Sulfur atoms as spheres for prominence
view.addStyle({'resn': 'CYS', 'atom': 'SG'}, {'sphere': {'radius': 0.4, 'color': '#fdcb6e'}})
view.addLabel("Disulfide Bridge", {'backgroundColor': '#e17055', 'fontColor': 'white'}, {'resn': 'CYS', 'atom': 'SG'})

view.zoomTo()
view.center()
view.show()

print("Educational Insight: Note how 'Cys1' and 'Cys6' residues are forced into proximity to satisfy the disulfide geometry.")

## 2. Case Study: Cyclosporine A (D-Amino Acid Macrocycle)

**Cyclosporine A** is a powerhouse drug used to prevent organ transplant rejection. It is an **11-mer cyclic peptide** that features multiple D-amino acids.

**The Challenge**: Head-to-tail cyclization requires the first residue to bond with the last residue. D-amino acids (mirror images of normal L-amino acids) are used to prevent the cycle from being broken down by the body.

In [ ]:
# We use a representative 11-mer with D-Alanine to show the cyclic capability
cyclosporine_proxy = "ALA-VAL-LEU-ILE-D-ALA-GLY-MET-PHE-PRO-LEU-VAL"

print("💍 Cyclizing 11-mer with D-amino acid integration...")

cyc_pdb = generate_pdb_content(
    sequence_str=cyclosporine_proxy,
    cyclic=True,       # Enable head-to-tail closure
    minimize_energy=True,
    cap_termini=False
)

print("✅ Cyclic Macrocycle Generated.")

In [ ]:
import py3Dmol
view = py3Dmol.view(width=800, height=500)
view.setBackgroundColor('#fdfdfd')
view.addModel(cyc_pdb, 'pdb')

# Style the Cyclic Backbone with high-contrast colors
view.setStyle({'model': -1}, {'cartoon': {'color': '#00b894', 'opacity': 0.8}})
view.setStyle({'model': -1}, {'stick': {'radius': 0.2, 'color': '#55efc4'}})

# Highlight the D-Alanine residue in Intense Red
view.setStyle({'resn': 'D-ALA'}, {'stick': {'radius': 0.5, 'color': '#d63031'}})
view.addLabel("D-Alanine Hinge", {'backgroundColor': '#d63031', 'fontColor': 'white'}, {'resn': 'D-ALA'})

view.zoomTo()
view.center()
view.show()

print("Educational Insight: The D-Alanine (red) provides a 'hinge' that helps stabilize the circular backbone.")

## 3. Finding the "Native" State (The Low-Energy Ensemble)

Macrocycles are highly constrained, but they can still "breathe". In drug discovery, we want the structure that is the most stable (lowest energy). We use **Batched Generation** to sample many possibilities and find the global minimum.

In [ ]:
n_peptides = 20
print(f"🔥 Sampling {n_peptides} possible conformations for our therapeutic lead...")

generator = BatchedGenerator(oxytocin_seq, n_batch=n_peptides, full_atom=True)
batch = generator.generate_batch(drift=10.0)

print("✅ Ensemble Generated.")
print("Educational Note: In a real lab, you would run MD simulations on these samples to find the one with the best binding affinity.")

In [ ]:
import py3Dmol
view = py3Dmol.view(width=800, height=450)
view.setBackgroundColor('#fdfdfd')

# Qualitative palette for the ensemble
colors = ['#6c5ce7', '#e17055', '#00b894', '#0984e3', '#fdcb6e']

for i in range(min(5, n_peptides)):
    view.addModel(batch.to_pdb(i), 'pdb')
    # Use thin "sticks" for an ultra-premium look across the ensemble
    view.setStyle({'model': i}, {'stick': {'radius': 0.15, 'color': colors[i % len(colors)], 'opacity': 0.8}})
    # Use addStyle for the spheres so they don't erase the side-chain sticks
    view.addStyle({'model': i, 'atom': 'SG'}, {'sphere': {'radius': 0.4, 'color': '#fab1a0'}})

view.zoomTo()
view.center()
view.show()


### 🏆 Experiment for the User
Try replacing `PRO` in the macrocycle with `D-PRO` and see if the ring closure becomes easier or harder. 🧪

**The molecules are yours to discover.** 🧪💍🧬